# Projeto Automação Web - Busca de Preços

### Objetivo: treinar um projeto em que a gente tenha que usar automações web com Selenium para buscar as informações que precisamos

- Já fizemos um projeto com esse objetivo no Módulo de Python e Web e em gravações de encontros ao vivo, mas não custa nada treinar mais um pouco.

### Como vai funcionar:

- Imagina que você trabalha na área de compras de uma empresa e precisa fazer uma comparação de fornecedores para os seus insumos/produtos.

- Nessa hora, você vai constantemente buscar nos sites desses fornecedores os produtos disponíveis e o preço, afinal, cada um deles pode fazer promoção em momentos diferentes e com valores diferentes.

- Seu objetivo: Se o valor dos produtos for abaixo de um preço limite definido por você, você vai descobrir os produtos mais baratos e atualizar isso em uma planilha.
- Em seguida, vai enviar um e-mail com a lista dos produtos abaixo do seu preço máximo de compra.

- No nosso caso, vamos fazer com produtos comuns em sites como Google Shopping e Buscapé, mas a ideia é a mesma para outros sites.

### Outra opção:

- APIs

### O que temos disponível?

- Planilha de Produtos, com os nomes dos produtos, o preço máximo, o preço mínimo (para evitar produtos "errados" ou "baratos de mais para ser verdade" e os termos que vamos querer evitar nas nossas buscas.

### O que devemos fazer:

- Procurar cada produto no Google Shopping e pegar todos os resultados que tenham preço dentro da faixa e sejam os produtos corretos
- O mesmo para o Buscapé
- Enviar um e-mail para o seu e-mail (no caso da empresa seria para a área de compras por exemplo) com a notificação e a tabela com os itens e preços encontrados, junto com o link de compra.

In [66]:
#passo a passo

# criar um navegador

# importar/visualizar a base de dados

# para cada item dentro da nossa base de dados, para cada produto

    # procurar esse produto no google shopping
        # verificar se algum dos produtos do google shopping está dentro da minha faixa de preço
    # procurar esse produto no buscapé
        # verificar se algum dos produtos do buscapé está dentro da minha faixa de preço
        
# salvar as ofertas boas em um dataframe (tabela)

# exportar pro excel

# enviar por email o resultado da tabela   
     

In [67]:
# criar um navegador
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import time
import pandas as pd

# criar o navegador
nav = webdriver.Chrome()

# importar/visualizar a base de dados
tabela_produtos = pd.read_excel("buscas.xlsx")
display(tabela_produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64 gb,mini watch,3000,3500
1,rtx 3060,zota galax,4000,4500


Definição das funções de busca no google e no buscape

In [68]:
# pesquisar pelo produto
def verificar_tem_termos_banidos(lista_termos_banidos, nome):
    tem_termos_banidos = False
    for palavra in lista_termos_banidos:
        if palavra in nome:
            tem_termos_banidos = True
    return tem_termos_banidos

def vereficar_tem_todos_termos_produtos(lista_termos_nome_produto, nome):
    tem_todos_termos_produto = True
    for palavra in lista_termos_nome_produto:
        if palavra not in nome:
            tem_todos_termos_produto = False
    return tem_todos_termos_produto


def busca_google_shopping(nav, produto, termos_banidos, preco_min, preco_max):
    
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(' ') # separa a string pelos espaços contidos nela para poder analisar cada um a parte
    lista_termos_nome_produto = produto.split(' ') # separa a string pelos espaços contidos nela para poder analisar cada um a parte
    lista_ofertas = []
    preco_min = float(preco_min)
    preco_max = float(preco_max)

    # entrar no google
    nav.get('https://www.google.com')

    nav.find_element('xpath', '//*[@id="APjFqb"]').send_keys(produto, Keys.ENTER)
    time.sleep(1)

    # entrar na aba shopping
    nav.find_element('xpath', '//*[@id="hdtb-sc"]/div/div/div[1]/div/div[2]/a/div').click()

    # clicar na aba shopping cquando tiver classes com o mesmo nome
    # elementos = nav.find_elements(By.CLASS_NAME, 'hdtb-mitem')
    # for item in elementos:
        # if "Shpping" in item.text:
            # item.click()
            # break # para interromper caso tenha outro elemento com o mesmo nome no for 

    # pegar as informações do produto
    lista_resultados = nav.find_elements('class name', 'i0X6df')

    for resultado in lista_resultados:
        
        nome = resultado.find_element('class name', 'tAxDx').text
        nome = nome.lower()
        # analisar se ele não tem nenhum termo banido
        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)
                
        
        # analisar se ele tem TODOS os termos do nome do produto 
        tem_todos_termos_produto = vereficar_tem_todos_termos_produtos(lista_termos_nome_produto, nome)
        
        try:        
            # selecionar só os elementos que tem tem_termos_banidos = False e ao mesmo tempo tem_todos_termos_produto = True
            if not tem_termos_banidos and tem_todos_termos_produto: # if tem_termos_banidos == False and tem_todos_termos_produto == True:
                preco = resultado.find_element('class name', 'a8Pemb').text
                preco = preco.replace('R$', '').replace(' ', '').replace('.', '').replace(',', '.') # tratamento de texto
                preco = float(preco) # pronto pra transformar em float
                
                # se o preco está entre o preco_min e o preco_max
                if preco_min <= preco <= preco_max:
                    elemento_referencia = resultado.find_element('class name', 'bONr3b')
                    elemento_pai = elemento_referencia.find_element('xpath', '..') # '..' -> pega o elemento parent (que vem logo antes) dele
                    link = elemento_pai.get_attribute('href')
                    lista_ofertas.append((nome, preco, link))
        except:
            continue
                
    return lista_ofertas

def busca_buscape(nav, produto, termos_banidos, preco_min, preco_max):
    # tratar valores
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(' ') # separa a string pelos espaços contidos nela para poder analisar cada um a parte
    lista_termos_nome_produto = produto.split(' ') # separa a string pelos espaços contidos nela para poder analisar cada um a parte
    lista_ofertas = []
    preco_min = float(preco_min)
    preco_max = float(preco_max)
    
    # buscar produto no buscape
    nav.get("https://www.buscape.com.br/")
    nav.find_element('xpath', '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(produto, Keys.ENTER)
    
    # pegar os resultados
    lista_resultados = nav.find_elements('class name', 'ProductCard_ProductCard_Inner__gapsh')
    
    for resultado in lista_resultados:
        preco = resultado.find_element('class name', 'Text_MobileHeadingS__HEz7L').text
        nome = resultado.find_element('class name', 'ProductCard_ProductCard_Name__U_mUQ').text
        nome = nome.lower()
        link = resultado.get_attribute('href')
        
        # analisar tem termo banido e tem todos os termos do produto
        # analisar se ele não tem nenhum termo banido
        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)
                
        # analisar se ele tem TODOS os termos do nome do produto 
        tem_todos_termos_produto = vereficar_tem_todos_termos_produtos(lista_termos_nome_produto, nome)
        
        try:
            # analisar se o preco esta entre o preco min e o preco max
            if not tem_termos_banidos and tem_todos_termos_produto:
                preco = preco.replace('R$', '').replace(' ', '').replace('.', '').replace(',', '.') # tratamento de texto
                preco = float(preco) # pronto pra transformar em float
                
                if preco_min <= preco <= preco_max:
                    lista_ofertas.append((nome, preco, link))
        except:
            continue
            
    # retomar a lista de ofertas do buscape
    return lista_ofertas

Contrução da nossa lista de ofertas encontradas

In [69]:
# salvar as ofertas boas em um dataframe (tabela)
tabela_ofertas = pd.DataFrame()

for linha in tabela_produtos.index:
    # pesquisar pelo produto
    produto = tabela_produtos.loc[linha, 'Nome']
    termos_banidos = tabela_produtos.loc[linha, 'Termos banidos']
    preco_min = tabela_produtos.loc[linha, 'Preço mínimo']
    preco_max = tabela_produtos.loc[linha, 'Preço máximo']

    lista_ofertas_google_shopping = busca_google_shopping(nav, produto, termos_banidos, preco_min, preco_max)
    if lista_ofertas_google_shopping:
        tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns=['produto', 'preco', 'link'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_google_shopping])
    else:
        tabela_google_shopping = None # para garantir que as tabelas sejam sempre redefinidas, sem juntar tabelas de iphones com placas de videos
        
    lista_ofertas_buscape = busca_buscape(nav, produto, termos_banidos, preco_min, preco_max)
    if lista_ofertas_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns=['produto', 'preco', 'link'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_buscape])
    else:
        tabela_buscape = None # para garantir que as tabelas sejam sempre redefinidas, sem juntar tabelas de iphones com placas de videos

display(tabela_ofertas)

,produto,preco,link
0,iphone 12 - verde - 64 gb,3111.00,https://www.google.com/url?url=https://reidoce...
1,"iphone 12 apple 64gb preto tela 6,1” 12mp ios",3110.00,https://www.google.com/url?url=https://shoppin...
2,"iphone 12 (branco, 64gb)",3499.00,https://www.google.com/url?url=https://www.geo...
3,iphone 12 64gb azul - excelente,3190.00,https://www.google.com/url?url=https://www.cdk...
4,usado: iphone 12 64gb preto excelente - trocafone,3019.00,https://www.google.com/url?url=https://www.pon...
5,usado: iphone 12 64gb verde excelente - trocafone,3019.00,https://www.google.com/url?url=https://www.ext...
0,smartphone apple iphone 12 vermelho 64gb câmer...,3134.99,https://www.buscape.com.br/celular/smartphone-...
0,placa de vídeo geforce rtx 3060 gaming oc 12gb...,4343.04,https://www.google.com/url?url=https://br.octo...
1,placa de vídeo msi rtx 3060ti twin fan oc edit...,4299.00,https://www.google.com/url?url=https://www.inf...


Exportar a base de ofertas para Excel

In [70]:
# exportar pro excel

tabela_ofertas.to_excel('Ofertas.xlsx', index=False)

Enviando o e-mail

In [ ]:
# enviar por e-mail o resultado da tabela
import win32com.client as win32

#verificando se existe alguma oferta dentro da tabela de ofertas
if len(tabela_ofertas.index) > 0:
    # vou enviar email
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = 'XXXX@gmail.com'
    mail.Subject = 'Produto(s) Encontrado(s) na faixa de preço desejada'
    mail.HTMLBody = f"""
    <p>Prezados,</p>
    <p>Encontramos alguns produtos em oferta dentro da faixa de preço desejada. Segue tabela com detalhes</p>
    {tabela_ofertas.to_html(index=False)}
    <p>Qualquer dúvida estou à disposição</p>
    <p>Att.,</p>
    """
    
    mail.Send()

nav.quit()  